In [1]:
import numpy as np
from wingstructure.structure import beammechanics as beam
%matplotlib ipympl

In [2]:
# creating test geometry
# idaflieg kennwerte HT-Kohlefasergewebe mit k = 0.5

geometry_section = np.load('contour.npy')

youngsmoduli_left = np.ones(len(geometry_section[0])-1)*12.5*10**9 # steel
youngsmoduli_center = np.ones(len(geometry_section[1])-1)*12.5*10*9
youngsmoduli_right = np.ones(len(geometry_section[2])-1)*12.5*10**9

shearmoduli_left = np.ones(len(geometry_section[0])-1)*19.6*10**9 # steel
shearmoduli_center = np.ones(len(geometry_section[1])-1)*19.6*10**9
shearmoduli_right = np.ones(len(geometry_section[2])-1)*19.6*10**9

thickness_left = np.ones(len(geometry_section[0])-1)*0.003
thickness_center = np.ones(len(geometry_section[1])-1)*0.003
thickness_right = np.ones(len(geometry_section[2])-1)*0.003

youngsmoduli_section = youngsmoduli_left, youngsmoduli_center, youngsmoduli_right
shearmoduli_section = shearmoduli_left, shearmoduli_center, shearmoduli_right
thickness_section = thickness_left, thickness_center, thickness_right

generate_testsection = lambda : beam.Crosssection(geometry_section, youngsmoduli_section, shearmoduli_section, thickness_section)
testsections = [generate_testsection()]*60

In [3]:
import pickle as pkl

with open('export.pkl', 'rb') as afile:

    y, c, c_l, c_m = pkl.load(afile)
    # position, chord of wing, lift coefficient, moment coefficient
    
y = y[59:119] # halfspan
c = c[59:118]
c_l = c_l[59:118] # last one not because of equal length with distance
c_m = c_m[59:118] # is there a better way?

ρ = 1.225
v = 60 # random
c = 0.632 # constant

In [4]:
distance = []
for i in range(0,len(y)-1):
    distance_i = y[i+1] - y[i]
    distance.append(distance_i)
    
lift = beam.lift(ρ, v, c_l, c)
moment = beam.moment(ρ, v, c_m, c)

Q = np.array((np.zeros(59), lift*distance)).T # shearforce
T = moment*distance # torsional moment

In [5]:
twistangle = []
for i in range(0,len(testsections)-1):
    twist_i = testsections[i].twist(Q[i],T[i])*distance[i]
    twistangle.append(twist_i)
    
twistangle
sum(twistangle)*180/np.pi

0.009758503451913779

In [6]:
geol = testsections[0].geometries[0]
geoc = testsections[0].geometries[1]
geor = testsections[0].geometries[2]

In [7]:
from matplotlib import pyplot as plt
center = testsections[0].shearcenter()
c4 = testsections[0].c4_point()

plt.plot(geol[:,0], geol[:,1])
plt.plot(geoc[:,0], geoc[:,1])
plt.plot(geor[:,0], geor[:,1])
plt.plot(center[0], center[1], 'o')
plt.plot(c4[0], c4[1], 'x')
plt.axis('equal')
center

FigureCanvasNbAgg()

array([-0.11967728, -0.01073985])

In [8]:
Q = [0,1]
T = 1
Q_new = beam.transform(np.array((Q)), testsections[0].Θ)

In [9]:
shearflow = testsections[0].shearflow_total(Q_new, T)
shearflow = np.array(shearflow)/30

In [13]:
plt.figure()
shearflow_plot = testsections[0].plot_around_profil(shearflow)

shearflow_plot_left = shearflow_plot[0]
shearflow_plot_center = shearflow_plot[1]
shearflow_plot_right = shearflow_plot[2]

plt.plot(shearflow_plot_left[:,0], shearflow_plot_left[:,1], 'orange')
plt.plot(shearflow_plot_center[:,0], shearflow_plot_center[:,1], 'orange')
plt.plot(shearflow_plot_right[:,0], shearflow_plot_right[:,1], 'orange')



center = testsections[0].shearcenter()
c4 = testsections[0].c4_point()

plt.plot(geol[:,0], geol[:,1])
plt.plot(geoc[:,0], geoc[:,1])
plt.plot(geor[:,0], geor[:,1])
plt.plot(center[0], center[1], 'o')
plt.plot(c4[0], c4[1], 'x')
plt.axis('equal')
center

line1 = np.array((shearflow_plot_left[0], shearflow_plot_right[0]))
plt.plot(line1[:,0], line1[:,1], 'orange')

line2 = np.array((shearflow_plot_left[-1], shearflow_plot_right[-1]))
plt.plot(line2[:,0], line2[:,1], 'orange')

FigureCanvasNbAgg()

In [14]:
def plotdistri(Qy, Qz, T):
    
    Q = [Qy,Qz]
    Q_new = beam.transform(np.array((Q)), testsections[0].Θ)
    shearflow = testsections[0].shearflow_total(Q_new, T)
    shearflow = np.array(shearflow)/30
    
    plt.clf()
    
    shearflow_plot = testsections[0].plot_around_profil(shearflow)

    shearflow_plot_left = shearflow_plot[0]
    shearflow_plot_center = shearflow_plot[1]
    shearflow_plot_right = shearflow_plot[2]
    
    center = testsections[0].shearcenter()
    c4 = testsections[0].c4_point()
    
    plt.plot(geol[:,0], geol[:,1])
    plt.plot(geoc[:,0], geoc[:,1])
    plt.plot(geor[:,0], geor[:,1])
    plt.plot(center[0], center[1], 'o')
    plt.plot(c4[0], c4[1], 'x')
    plt.axis('equal')
    
    
    plt.plot(shearflow_plot_left[:,0], shearflow_plot_left[:,1], 'orange')
    plt.plot(shearflow_plot_center[:,0], shearflow_plot_center[:,1], 'orange')
    plt.plot(shearflow_plot_right[:,0], shearflow_plot_right[:,1], 'orange')

    plt.arrow(c4[0], c4[1], 0, -Qz/10, linewidth=2, color='red')
    plt.arrow(c4[0], c4[1], -Qy/10, 0, linewidth=2, color='red')



    line1 = np.array((shearflow_plot_left[0], shearflow_plot_right[0]))
    plt.plot(line1[:,0], line1[:,1], 'orange')

    line2 = np.array((shearflow_plot_left[-1], shearflow_plot_right[-1]))
    plt.plot(line2[:,0], line2[:,1], 'orange')

In [17]:
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider
import ipywidgets as widgets

plt.figure()


Q_y_slider = FloatSlider(value=0.0, min=-5, max=5)
Q_z_slider = FloatSlider(value=0.0, min=-5, max=5)
T_slider = FloatSlider(value=0.0, min=-5, max=5)

interact(plotdistri, Qy=Q_y_slider, Qz=Q_z_slider, T=T_slider)

FigureCanvasNbAgg()

interactive(children=(FloatSlider(value=0.0, description='Qy', max=5.0, min=-5.0), FloatSlider(value=0.0, desc…

<function __main__.plotdistri(Qy, Qz, T)>

In [16]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in range(0,len(y)):

    a = np.ones(len(testsections[i].geometries[0]))*y[i]
    #b = ones(len(testsections[i].geometries[1]))*y[i]
    c = np.ones(len(testsections[i].geometries[2]))*y[i]

    ax.plot(a, testsections[i].geometries[0][:,0], testsections[i].geometries[0][:,1], 'orange')
    #ax.plot(b, testsections[i].geometries[1][:,0], testsections[i].geometries[1][:,1], 'orange')
    ax.plot(c, testsections[i].geometries[2][:,0], testsections[i].geometries[2][:,1], 'orange')




FigureCanvasNbAgg()